## PART 0: Reviewing Earlier Steps



As almost all the "useful" journals are identified for Q1 100K run, it is necessary to understand & review the Scimago datasets and the earlier functions. As it was seen in the previous steps, there are different versions (both online & offline), (two journals merged into one after a certain year) of some journals, it important to create a final eligible journals dataset to be merged with the date results. 

Another factor is that, during complete_one runs there are 30K+ articles that have completed the complete pipeline and ready to be examined. These articles are currenty stored in date_df & date_df_second_run datasets. These journals must be utilized for maximum efficiency and minimizing unnecessary time loss. 

In [1]:
import pandas as pd
import pickle 
import numpy as np

from quartile import *


#from scihub_upgraded import SciHub
from crossref.restful import Works, Etiquette

#import warnings
#warnings.filterwarnings("ignore")

b'Skipping line 32712: expected 20 fields, saw 21\nSkipping line 32713: expected 20 fields, saw 21\n'
b'Skipping line 20760: expected 20 fields, saw 21\nSkipping line 22914: expected 20 fields, saw 21\n'
b'Skipping line 19393: expected 20 fields, saw 21\nSkipping line 19394: expected 20 fields, saw 21\n'
b'Skipping line 33757: expected 20 fields, saw 21\n'
b'Skipping line 19053: expected 20 fields, saw 21\nSkipping line 20953: expected 20 fields, saw 21\n'


In [9]:
%store -r retr_onetwo

After analysis, it is seen that:

* scimago.py -> is responsible for "cat_sbj_dict" that is used in quartile.py for "Category" to "Subj. Area" matching.

* quartile.py -> is responsible for transforming Scimago excels into Q1-4 journal datasets. It also splits the journals based on their "highest Q'd" subj. areas, and splits their Total Docs. accordingly. As the remainder is distributed randomly, each run of this file generates slightly different doc. count for journals with "multimode"d subj areas. However, this should not effect the results greatly. The code also looks OK and efficient except for an .iterrows() loop, which can be improved but is ignored as it is not a huge performance or time issue.

* As Unpy is removed from the overall pipeline, all Unpy related code is also shelved.

* scimago_upgraded -> is responsible for the whole get_dates loop as the entire PDF & date extraction is now executed via SciHub.


The biggest improvement area while we are waiting for Q1 useful journals to be completed are making sure that the final journal list is compatible with quartile.py flow.





In [10]:
# Get q_df(s):
q_df = quar_df_creator(1)

b'Skipping line 32712: expected 20 fields, saw 21\nSkipping line 32713: expected 20 fields, saw 21\n'
b'Skipping line 20760: expected 20 fields, saw 21\nSkipping line 22914: expected 20 fields, saw 21\n'
b'Skipping line 19393: expected 20 fields, saw 21\nSkipping line 19394: expected 20 fields, saw 21\n'
b'Skipping line 33757: expected 20 fields, saw 21\n'
b'Skipping line 19053: expected 20 fields, saw 21\nSkipping line 20953: expected 20 fields, saw 21\n'


In [11]:
# Create q_jrnl & q_pivor using q_df & article count target:

q_jrnl, q_pivot = retr_ready(q_df,100000)

In [12]:
q_ready = q_jrnl.groupby([q_jrnl["Issn"].map(tuple),q_jrnl["Year"]])["Total_Docs"].sum().reset_index()


In [17]:
retr_onetwo.Total_Docs.equals(q_ready.Total_Docs)

True

In [4]:
q_jrnl[q_jrnl.Issn.map(lambda x: any([iss.startswith(" ") for iss in x]))]

,Issn,Year,Sbj_Area,Total_Docs


VERY IMPORTANT CODE BELOW:

In [61]:
q_dftesto.apply(lambda x: x["Issn"]+ (x["Year"],), axis=1).value_counts()

(13474065,  00214922, 2011)    2
(13474065,  00214922, 2010)    2
(00346861,  15390756, 2010)    1
(15384721, 2017)               1
(10902112,  10499644, 2017)    1
                              ..
(17453062,  17453054, 2013)    1
(17543266,  17543274, 2013)    1
(20728050,  02599422, 2013)    1
(0046385X, 2013)               1
(02572117, 2020)               1
Length: 75528, dtype: int64

In [68]:
q_dftesto[q_df.Issn.map(lambda x: "13474065" in x)][["Issn","Total_Docs"]]

,Issn,Total_Docs
6712,"(13474065, 00214922)",2
6864,"(13474065, 00214922)",1737
6718,"(13474065, 00214922)",2
7018,"(13474065, 00214922)",1758
8219,"(13474065, 00214922)",1741
8586,"(13474065, 00214922)",1496
9327,"(13474065, 00214922)",1451
9262,"(13474065, 00214922)",1174
9998,"(13474065, 00214922)",1418
10566,"(13474065, 00214922)",1275


In [63]:
q_ready[q_ready.Issn.map(lambda x:"13474065" in x)]

,Issn,Year,Total_Docs
31196,"(13474065, 00214922)",2010,1739
31197,"(13474065, 00214922)",2011,1760
31198,"(13474065, 00214922)",2012,1741
31199,"(13474065, 00214922)",2013,1496
31200,"(13474065, 00214922)",2016,1451
31201,"(13474065, 00214922)",2017,1174
31202,"(13474065, 00214922)",2018,1418
31203,"(13474065, 00214922)",2019,1275


The only USEFUL finding in this step is that, retr_complete_one has a slightly altered Issn column with fewer ISSN codes than a newly created version of it (q_ready).

As a result, this column will be REPLACED once all the work in the previous nb is completed!

## PART 1: Creating a COMPELETE (1st + 2nd run) date_df

In [4]:
%store -r date_df
%store -r second_run_date_df

In [24]:
duplicate_dois = set(date_df.index.tolist()).intersection(set(second_run_date_df.index.tolist()))
date_df.drop(duplicate_dois, inplace=True)

date_df_full = date_df.append(second_run_date_df,verify_integrity=True)
date_df_full.drop("Year", axis=1, inplace=True)

date_df_full is created!

In [243]:
%store date_df_full

Stored 'date_df_full' (DataFrame)


In [244]:
with open("date_df_full","wb") as fp:
    pickle.dump(date_df_full, fp)

The analysis below shows that, the missing (.) in the regex results in a 0.8% percent found date increase. This alone is not necessary for re-doing the entire dataset BUT:

* we now have the direct URLs for the successful articles, which should quicken & simplify the process if we decide to re-run
* KEYWORDS extraction might be necessary for future anaylsis, which would give us a second chance to loop through anyways!

In [46]:
sh = SciHub()
new_date_regex_test = date_df_full[date_df_full.Results.map(len)==2].sample(1000)
new_date_regex_test["New_Results"] = new_date_regex_test.index.map(sh.get_dates)
new_date_regex_test["New_Results"].map(len).value_counts()

2     990
3       8
21      1
13      1
Name: New_Results, dtype: int64

### PART 1.A: date_df_full_analysis

TTD for date_df_full:

* "skipped"  -> not filtered BUT in date_df_str_fail DONE(!)
* "published" column function -> DONE!
* "Results" column expand function -> DONE!
* "language" analysis (can we extact the lang. from the title for non-en articles) -> can't extract dates if not in Eng. so will be DROPPED & DONE!
* Check "ISSN" format 
* Simplify "author" column -> maybe create a author_df -> DONE!

In [242]:
date_df_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 128374 entries, 10.1002/aic.12400 to 10.1177/0096340214539115
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   reference-count         128374 non-null  int64 
 1   publisher               128371 non-null  object
 2   published-print         120056 non-null  object
 3   is-referenced-by-count  128374 non-null  int64 
 4   title                   128374 non-null  object
 5   author                  120278 non-null  object
 6   published-online        82614 non-null   object
 7   reference               89462 non-null   object
 8   container-title         128374 non-null  object
 9   language                57136 non-null   object
 10  issued                  128374 non-null  object
 11  ISSN                    128374 non-null  object
 12  subject                 128013 non-null  object
 13  published               62215 non-null   object
 14  Results

In [95]:
# 1- "not_in Scihub" FIX (date_df_full):

date_df_full.loc[date_df_full.Results.map(type)==str, "Results"] = date_df_full[date_df_full.Results.map(type)==str]["Results"].str.replace("article_not_in Scihub","article_not_in_Scihub")

# STORED & DUMPED AFTER THIS

date_df_full[date_df_full.Results.map(type)==str].Results.value_counts()

article_not_in_Scihub    9485
skipped                  3868
cant_read_pdf             242
direct_url_error           10
pdf_bytes_error             7
Name: Results, dtype: int64

In [96]:
date_df_full[date_df_full.Results.map(type)==list].Results.map(len).value_counts()

2    78218
3    36495
Name: Results, dtype: int64

In [231]:
# 2- SPLITTING THE DATASETS (date_df_ok):

date_df_str_fail = date_df_full[date_df_full.Results.map(type)==str].copy()

date_df_ok = date_df_full[date_df_full.Results.map(type)==list].copy()

In [232]:
# 3- "RESULTS" COLUMN ENGINEERING:
def result_mapper(lst):
    if len(lst) == 2:
        return lst[0], np.nan, np.nan
    elif len(lst) == 3:
        return lst[0], lst[1], lst[2]

date_df_ok["direct_url"], date_df_ok["Rec_date"], date_df_ok["Acc_date"] = zip(*date_df_ok.Results.map(result_mapper))

In [233]:
date_df_ok["direct_url"].map(type).value_counts()

<class 'str'>    114759
Name: direct_url, dtype: int64

In [234]:
date_df_ok["Rec_date"].map(type).value_counts()

<class 'float'>       78245
<class 'str'>         34050
<class 'NoneType'>     2464
Name: Rec_date, dtype: int64

In [235]:
date_df_ok["Acc_date"].map(type).value_counts()

<class 'float'>       78245
<class 'str'>         30891
<class 'NoneType'>     5623
Name: Acc_date, dtype: int64

In [236]:
# 4- "PUBLISHED" COLUMN ENGINEERING:

date_df_ok.loc[(date_df_ok.published.isna()) & (date_df_ok["published-print"].isna()) & (date_df_ok["published-online"].notna()),"published"] = date_df_ok.loc[(date_df_ok.published.isna()) & (date_df_ok["published-print"].isna()) & (date_df_ok["published-online"].notna()),"published-online"]
date_df_ok.loc[(date_df_ok.published.isna()) & (date_df_ok["published-print"].notna()) & (date_df_ok["published-online"].isna()),"published"] = date_df_ok.loc[(date_df_ok.published.isna()) & (date_df_ok["published-print"].notna()) & (date_df_ok["published-online"].isna()),"published-print"]

def earlier_date_part(row):
    pp_year = row["published-print"]["date-parts"][0][0]
    po_year = row["published-online"]["date-parts"][0][0]
       
    if pp_year<po_year:
        return row["published-print"]
    elif po_year<pp_year:
        return row["published-online"]
    else:
        try:
            pp_month = row["published-print"]["date-parts"][0][1]
        except IndexError:
            pp_month = 12
        try:
            po_month = row["published-online"]["date-parts"][0][1]
        except IndexError:
            po_month = 12
        if pp_month<po_month:
            return row["published-print"]
        elif po_month<pp_month:
            return row["published-online"]
        else:
            return row["published-online"]


date_df_ok.loc[date_df_ok.published.isna(),"published"] = date_df_ok[date_df_ok.published.isna()].apply(earlier_date_part, axis=1)

In [237]:
# 5- "REFERENCE" & "LANGUAGE" COLUMN ANALYSIS:

# It seems that both columns are not useful, and will be dropped. They can be retrieved from date_df_full in the future if necessary.

date_df_ok.drop(["reference","language"], axis=1, inplace=True)

In [238]:
# 6- "AUTHOR" COLUMN ANALYSIS:

date_df_ok.loc[date_df_ok.author.notna(),"first_author"] = date_df_ok[date_df_ok.author.notna()].author.map(lambda x: x[0])

In [240]:
%store date_df_ok

Stored 'date_df_ok' (DataFrame)


In [241]:
with open("date_df_ok","wb") as fp:
    pickle.dump(date_df_ok, fp)

### PART 1.B: retr_complete_one & second_run_df analysis

In [19]:
%store -r retr_complete_one
%store -r second_run_df

In [20]:
retr_complete_one.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74421 entries, 0 to 74420
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Issn           74421 non-null  object
 1   Year           74421 non-null  int32 
 2   Total_Docs     74421 non-null  int64 
 3   Sample_Count   74421 non-null  int64 
 4   CrossRef_retr  74421 non-null  object
 5   DOI            74421 non-null  object
 6   Results        64119 non-null  object
dtypes: int32(1), int64(2), object(4)
memory usage: 4.3+ MB


There are 3 dataframes to work on:

* no need for DOI info in any of them
* no need for "Sample_Count"

1- second_run_df:
* replace Second_CR_retr with CrossRef_retr
* get True & False from "2nd_Results"

2- retr_compete_one:
* drop all included in other 2
* get True & False from "Results" 

3- missing49:
* loop through CR + SH get_dates

In [ ]:
missing49 = retr_complete_one[(retr_complete_one.CrossRef_retr == True) & (retr_complete_one.Results.map(type)==float)].copy()

In [136]:
# 1- second_run_df:

second_run_df["SH_Results"] = second_run_df["2nd_Results"].map(lambda x: True if len(x)>0 else False)
second_run_df.drop(["Sample_Count","CrossRef_retr","DOI","Results","2nd_Results"], axis=1, inplace=True)
second_run_df.rename(columns={"Second_CR_retr":"CR_Results"},inplace=True)

In [ ]:
# 2- retr_complete_one:
retr_complete_one.drop(second_run_df.index.tolist(), inplace=True)
retr_complete_one.drop(missing49.index.tolist(), inplace=True)
retr_complete_one["SH_Results"] = True
retr_complete_one.drop(["Sample_Count","DOI","Results"], axis=1, inplace=True)
retr_complete_one.rename(columns={"CrossRef_retr":"CR_Results"},inplace=True)

In [ ]:
# 3- missing49:
missing49.drop(["Sample_Count","DOI","Results"], axis=1, inplace=True)
missing49.rename(columns={"CrossRef_retr":"CR_Results"}, inplace=True)

In [164]:
# 3- missing49 cont.:

from scihub_upgraded import SciHub
from crossref.restful import Works, Etiquette

my_etiquette = Etiquette('Analysing Publishing Delay in Academic Journals', 'v2.0', 'https://github.com/Spidey0023/THEsis-Codes', 'oguzkokes@gmail.com')
ogi_works = Works(etiquette=my_etiquette)

sh = SciHub()

# The full loop code for missing49:

issn_rry = missing49.Issn.values
year_rry = missing49.Year.values

#second_run_df.reset_index(drop=True, inplace=True)

miss49_meta_list = []
miss49_sh_date_dict = {}
useful_cols = ['DOI', 'references-count', 'publisher', 'published-print', 'is-referenced-by-count', 'title', 'author', 'published-online', 'reference', 'container-title', 'issued', 'ISSN', 'subject']

for i in range(len(missing49)):
    issn = issn_rry[i]
    year = year_rry[i]
    loop_meta = [w for w in ogi_works.filter(issn=issn, from_pub_date=year, until_pub_date=year, type="journal-article").sample(1).select(useful_cols)] 

    if len(loop_meta) == 0:
        missing49.loc[missing49.index[i],"Second_CR_retr"] = False
    else:
        missing49.loc[missing49.index[i],"Second_CR_retr"] = True
        missing49.loc[missing49.index[i],"DOI"] = loop_meta[0]["DOI"]
        miss49_sh_date_dict[loop_meta[0]["DOI"]] = sh.get_dates(loop_meta[0]["DOI"])

    miss49_meta_list.extend(loop_meta)



In [176]:
# 3- missing49 cont.:

missing49["Results"] = missing49.DOI.map(miss49_sh_date_dict)
missing49["SH_Results"] = missing49.Results.map(len) == 3
missing49.drop(["Second_CR_retr","DOI","Results"], axis=1, inplace=True)

49

In [187]:
# Concat all 3 journal related datasets: retr_onetwo

retr_onetwo = pd.concat([second_run_df,retr_complete_one, missing49], verify_integrity=True)

In [189]:
%store retr_onetwo

Stored 'retr_onetwo' (DataFrame)


In [245]:
with open("retr_onetwo","wb") as fp:
    pickle.dump(retr_onetwo, fp)

### PART 1.C: Add articles from missing49 to date_df_full & date_df_str_fail & date_df_ok

In [192]:
miss49_df = pd.DataFrame(miss49_meta_list)
miss49_df.set_index("DOI",drop=True,inplace=True)
miss49_df["Results"] = miss49_df.index.map(miss49_sh_date_dict)
miss49_df.rename(columns={"references-count":"reference-count"}, inplace=True)

date_df_full = pd.concat([date_df_full,miss49_df],verify_integrity=True)

In [ ]:
#Rather than looping again, date_df_ok & date_df_str_fail will be created again from date_df_full by going through the steps in the previous chapter. All datasets are SAVED & STORED again afterwards.

date_df_ok

retr_one_two

## PART 2: retr_onetwo

While working on date_df upgrades ona seperate nb, it would be time efficient to get any long runs on retr_onetwo out of the way. There are currently 2 improvement areas:

1- replace retr_onetwo ISSN column with q_ready's ISSN column which has more ISSN numbers attached.

2- to get CR_count fror journals, so that the Sample_count can be calculated accordingly. Because if a jrnl has less articles in CrossRef than Scimago Total Docs. it may cause issues during sampling. This will be done entirely with SumCetCet looping.

--- 

After some analysis regarding q_df, q_ready processes, it is seen that everything is in order & the number disperency is caused by duplicated journals as expected. The steps above can be executed. 

#### AS A BACKUP, retr_onetwo_oldissns will be pickled & dumped if any unforseen errors occur with the new ISSN column.

In [13]:
retr_onetwo_oldissns = retr_onetwo.copy()

with open("retr_onetwo_oldissns","wb") as fp:
    pickle.dump(retr_onetwo_oldissns, fp)

In [16]:
retr_onetwo.sort_index(inplace=True)

retr_onetwo["Issn"] = q_ready.Issn.map(list)

2    52249
1    21983
3      189
Name: Issn, dtype: int64

In [ ]:
%store retr_onetwo


retr_onetwo is now updated, STORED & DUMPED. We can create the SumCet loop for CR_Count

In [32]:
retr_onetwo

,Issn,Year,Total_Docs,CR_Results,SH_Results
0,"[00011541, 15475905]",2010,294,True,True
1,"[00011541, 15475905]",2011,315,True,False
2,"[00011541, 15475905]",2012,347,True,False
3,"[00011541, 15475905]",2013,422,True,True
4,"[00011541, 15475905]",2014,359,True,True
...,...,...,...,...,...
74416,"[8756758X, 14602695]",2017,170,True,True
74417,"[8756758X, 14602695]",2018,199,True,True
74418,"[8756758X, 14602695]",2019,210,True,True
74419,"[8756758X, 14602695]",2020,221,True,True


In [ ]:
# Full Loop for CR_Count: - DID NOT RUN & NO OGI VERSION!

my_etiquette = Etiquette('Analysing Publishing Delay in Academic Journals', 'v2.0', 'https://github.com/Spidey0023/THEsis-Codes', 'oguzkokes@gmail.com')
ogi_works = Works(etiquette=my_etiquette)

issn_rry = retr_onetwo.Issn.values
year_rry = retr_onetwo.Year.values

for i in range(len(retr_onetwo)):
    issn = issn_rry[i]
    year = year_rry[i]
    jrnl_count = ogi_works.filter(issn=issn, from_pub_date=year, until_pub_date=year, type="journal-article").count() 

    retr_onetwo.loc[retr_onetwo.index[i],"CR_Count"] = jrnl_count


with open("retr_onetwo","wb") as fp:
    pickle.dump(retr_onetwo, fp)

In [ ]:
# SumCet Loop for CR_Count:

cet_retr_onetwo_cr_count = retr_onetwo[:40000]
sum_retr_onetwo_cr_count = retr_onetwo[40000:]

issn_rry = cet_retr_onetwo_cr_count.Issn.values
year_rry = cet_retr_onetwo_cr_count.Year.values

for i in range(len(retr_onetwo)):
    issn = issn_rry[i]
    year = year_rry[i]
    jrnl_count = ogi_works.filter(issn=issn, from_pub_date=year, until_pub_date=year, type="journal-article").count() 

    cet_retr_onetwo_cr_count.loc[cet_retr_onetwo_cr_count.index[i],"CR_Count"] = jrnl_count


with open("cet_retr_onetwo_cr_count","wb") as fp:
    pickle.dump(cet_retr_onetwo_cr_count, fp)

SumCet loop is completed! The results are analysed below:

In [8]:
with open("sum_retr_onetwo_cr_count", "rb") as fp:
    retr_onetwo_sum = pickle.load(fp)

In [20]:
with open("cet_retr_onetwo_cr_count", "rb") as fp:
    retr_onetwo_cet = pickle.load(fp)

In [35]:
print(f"Number of jrnls with less than 50% artcls available on CrossRef (cet): {len(retr_onetwo_cet[(retr_onetwo_cet.CR_Count<retr_onetwo_cet.Total_Docs*.5) & (retr_onetwo_cet.CR_Results==True)])}")

print(f"Number of jrnls with less than 50% artcls available on CrossRef (sum): {len(retr_onetwo_sum[(retr_onetwo_sum.CR_Count<retr_onetwo_sum.Total_Docs*.5) & (retr_onetwo_sum.CR_Results==True)])}")

Number of jrnls with less than 50% artcls available on CrossRef (cet): 1273
Number of jrnls with less than 50% artcls available on CrossRef (sum): 1305


In [41]:
print(f" Jrnls with CR_Results but CR_Counts == 0 (cet): {len(retr_onetwo_cet[(retr_onetwo_cet.CR_Results) & (retr_onetwo_cet.CR_Count == 0)])}")

print(f" Jrnls with CR_Results but CR_Counts == 0 (sum): {len(retr_onetwo_sum[(retr_onetwo_sum.CR_Results) & (retr_onetwo_sum.CR_Count == 0)])}")

 Jrnls with CR_Results but CR_Counts == 0 (cet): 0
 Jrnls with CR_Results but CR_Counts == 0 (sum): 6


In [42]:
# The problematic jrnls from sum_run:
retr_onetwo_sum[(retr_onetwo_sum.CR_Results) & (retr_onetwo_sum.CR_Count == 0)]

,Issn,Year,Total_Docs,CR_Results,SH_Results,CR_Count
43578,"[15251403, 10947159]",2015,275,True,True,0.0
43579,"[15251403, 10947159]",2016,170,True,True,0.0
43580,"[15251403, 10947159]",2017,111,True,True,0.0
43581,"[15251403, 10947159]",2018,115,True,True,0.0
68297,"[20458932, 20458940]",2014,77,True,True,0.0
68298,"[20458932, 20458940]",2015,80,True,True,0.0


Concating back to retr_one_two & STORING & DUMPing

The old version is already backed-up as "retr_onetwo_old_issns"

In [43]:
retr_onetwo = pd.concat([retr_onetwo_cet,retr_onetwo_sum],verify_integrity=True)
retr_onetwo["CR_Count"] = retr_onetwo.CR_Count.map(int)

In [51]:
# The final version of retr_onetwo:
retr_onetwo.head()

,Issn,Year,Total_Docs,CR_Results,SH_Results,CR_Count
0,"[00011541, 15475905]",2010,294,True,True,351
1,"[00011541, 15475905]",2011,315,True,False,361
2,"[00011541, 15475905]",2012,347,True,False,259
3,"[00011541, 15475905]",2013,422,True,True,343
4,"[00011541, 15475905]",2014,359,True,True,376


In [48]:
%store retr_onetwo

with open("retr_onetwo","wb") as fp:
    pickle.dump(retr_onetwo, fp)

Stored 'retr_onetwo' (DataFrame)


retr_onetwo is now ready for 100K run. HOWEVER, before the 100K run, there are upgrades that needs to be completed on date_df_ok. These are:

1- re-check DATE info with the new regex patterns

2- create & get AFFILIATION info from PDFs
* Get & score:
    * E-mail
    * Uni name
    * Country name 

3- get ABSTRACT from articles or PDFs

This notebook will be paused again, to work on the upgrades & will be continued once everything is ready for the "final" & third date_df run. Based on the new info, will create retr_three & date_df_three. 

retr_three will be used for completing 100K run & date_df_three will be concat'd with the new articles and the whole process will be completed, reaedy for analysis!!1

---

### WE ARE BACK!

The upgrade tests are completed! The next steps for the project are as follows:

1 - run sh-wartortle on date_df_ok 

    * re-check dates

    * get affiliation

    * get keywords

2 - calculate sample_count for each jrnl in retr_onetwo using q_matrix & sbj_areas

    * artciles in data_df_ok should be excluded

3 - run CR loop to get DOIs + metadata for sample_count

4 - create & run sh_blastoise on the new artcl dataset

5- 100K artcl dataset is completed!


--- 

This nb will stay as a hub for part 5, will start step 1 in nb 5b